In [10]:
import requests
from bs4 import BeautifulSoup
import csv

base_url = "https://spotifycharts.com/regional-global-weekly/"
date_range = range(1, 5)  # for the four weeks of February
song_data = []

for week in date_range:
    day = 7*(week-1)+2
    if day<10:
      day=f"0{day}"
    url = base_url + f"2023-02-{day}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    chart_data = soup.find_all('tr')[1:]  # skip the header row
    for row in chart_data:
        rank = row.find(class_='chart-table-position').text.strip()
        title = row.find(class_='chart-table-track').find('strong').text.strip()
        artist = row.find(class_='chart-table-track').find('span').text.strip().split('by ')[-1]
        spotify_id = row.find(class_='chart-table-image')['data-src'].split(':')[-1].split('.')[0]
        record_company = row.find(class_='chart-table-track').find_all('span')[-1].text.strip()
        peak_rank = row.find(class_='chart-table-top-spot').text.strip()
        prev_rank = row.find(class_='chart-table-change').text.strip()
        streams = row.find(class_='chart-table-streams').text.strip()

        song_data.append({
            'rank': rank,
            'title': title,
            'artist': artist,
            'spotify_id': spotify_id,
            'record_company': record_company,
            'peak_rank': peak_rank,
            'prev_rank': prev_rank,
            'streams': streams
        })

# write the song data to a CSV file
with open('top_songs_feb_2.csv', mode='w', newline='', encoding='utf-8') as file:
    fieldnames = ['rank', 'title', 'artist', 'spotify_id', 'record_company', 'peak_rank', 'prev_rank', 'streams']
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    for song in song_data:
        writer.writerow(song)
